# Background Researcher

In [1]:
from kruppe.llm import OpenAILLM
from kruppe.data_source.news.nyt import NewYorkTimesData

# define  llm
llm = OpenAILLM()

# define news source
nyt = NewYorkTimesData(headers_path="/Users/danielliu/Workspace/fin-rag/.nyt-headers.json")

## Librarian

In [2]:
from kruppe.algorithm.agents import Librarian

librarian = Librarian(llm=llm, news_source=nyt)

In [3]:
resource_requests = await librarian.choose_resource("What are companies are affected by Trump's tax plan?")
resource_requests

[{'func_name': 'news_search',
  'parameters': {'query': 'Trump tax plan companies affected',
   'num_results': 5,
   'sort': 'relevance'},
  'purpose': "To identify and gather relevant news articles about which companies are impacted by Trump's tax plan.",
  'rank': '1'},
 {'func_name': 'news_recent',
  'parameters': {'days': 30,
   'num_results': 5,
   'filter': {'include': 'Trump tax plan', 'exclude': ''}},
  'purpose': 'To get recent news articles that might provide insight on companies affected by the tax plan.',
  'rank': '2'},
 {'func_name': 'news_archive',
  'parameters': {'start_date': '2016-01-01',
   'end_date': '2023-10-01',
   'num_results': 5,
   'filter': {'include': 'Trump tax plan', 'exclude': ''}},
  'purpose': "To find historical news articles that discuss the implications of Trump's tax plan on companies over time.",
  'rank': '3'}]

In [5]:
async for doc in librarian.retrieve(resource_requests):
    print(doc)

Async scraping links:  20%|██        | 1/5 [00:00<00:00,  9.62it/s]

Text: Advertisement
By JESSE DRUCKER and AUDREY CARLSEN DEC. 22, 2017

President Trump would save about $11 million on his taxes, if the new Republican tax overhaul were applied to his 2005 tax return, a New York Times analysis has found. The savings would be a roughly 30 percent cut. He would also save another $4.4 million on his eventual estate tax bill.
On Friday, President Trump signed the sweeping tax overhaul approved earlier this week by Congress, a package of cuts that would overwhelmingly benefit the wealthy.
The analysis is based in part on information from his 2005 federal tax return, the most recent available, publicly released in March by David Cay Johnston on the website DCReport.org . Breaking with decades of tradition by previous presidents, Mr. Trump has refused to make any of his tax returns public. The analysis compares what his tax burden would be under current law with what it would be under the new legislation.
On Tuesday, White House spokeswoman Sarah Huckabee Sa

Async scraping links: 100%|██████████| 5/5 [00:00<00:00, 18.32it/s]

Text: Trump Administration
Advertisement
Supported by
By Peter Baker
HARRISBURG, Pa. — President Trump took his campaign for tax cuts to another key battleground state on Wednesday in search of elusive Democratic votes as he argued that his plan would benefit middle-class Americans, like truck drivers, more than it would help his fellow business tycoons.
At a rally in an airport hangar reminiscent of his fiery events during last year’s presidential campaign, Mr. Trump told hundreds of truckers that slashing rates would produce more hauling contracts and put more money in their pockets. Uncertain that he can count on a fragile Republican majority during a critical Senate vote next week , he sought to pressure Pennsylvania’s Democratic senator to support the plan.
“We’re going to fight, and we’re going to get those Republicans and maybe of few of those Democrats to raise their hand,” Mr. Trump told the cheering crowd. “And you’re going to have so much money to spend in this wonderful cou

## Background Researcher

In [3]:
from kruppe.algorithm.background_module.research import BackgroundResearcher

query = "How will Nvidia’s recent GTC Conference announcements—specifically its new AI products and strategies—impact its competitive positioning and long-term growth in the evolving AI and data center markets?"
researcher = BackgroundResearcher(query=query, llm=llm, librarian=librarian)

In [5]:
info_requests = await researcher.create_info_requests()
info_requests

["To comprehensively answer the query, I want to know the specific details of Nvidia’s announcements at the recent GTC Conference, particularly regarding the new AI products and strategies introduced. Understanding the nature and specifications of these products will help gauge their potential effectiveness in the market and how they compare to existing offerings from competitors, providing insight into Nvidia's innovation trajectory and product differentiation.",
 "I also want to gather information on the current competitive landscape in the AI and data center markets, including key players, their market shares, and recent advancements. This context is crucial to understand how Nvidia's announcements position the company relative to competitors like AMD, Intel, and other startups, which will inform the assessment of Nvidia's competitive advantage and growth potential.",
 'Furthermore, it is essential to research market trends and projections for AI and data center segments over the ne

In [6]:
await researcher.retrieve_and_analyze(info_requests)
researcher.history

Async scraping links: 100%|██████████| 10/10 [00:28<00:00,  2.84s/it]


  0%|          | 0/5 [04:21<?, ?it/s]


CancelledError: 

In [9]:
len(researcher.history)

67

In [10]:
from kruppe.prompts.background import COMPILE_REPORT_USER, RESEARCH_STANDARD_SYSTEM


history = researcher.history
user_message = COMPILE_REPORT_USER.format(
    query=query,
    analyses="\n".join([item["analysis"] for item in history])
)

messages = [
    {"role": "system", "content": RESEARCH_STANDARD_SYSTEM},
    {"role": "user", "content": user_message},
]

llm_response = await llm.async_generate(messages)
llm_string = llm_response.text

llm_string

"**Background Report: Nvidia’s GTC Conference Announcements and Impact on Competitive Positioning and Long-Term Growth**\n\n**1. Overview of Nvidia's Position in AI and Data Center Markets:**\nNvidia is a leading player in the field of artificial intelligence (AI) and data processing, known for its advancements in graphics processing units (GPUs) that are increasingly being leveraged in AI applications. The company's technology is foundational for various sectors, including automotive, healthcare, data centers, and gaming, positioning Nvidia as a pivotal contributor to the growth and evolution of AI-driven solutions.\n\n**2. The Context of the GTC Conference:**\nThe GPU Technology Conference (GTC) has evolved from a specialized academic event into a major industry platform spotlighting AI innovations and advancements. Announcements made during the latest conference often set forth Nvidia's strategic directions and highlight new AI product offerings that can shape market dynamics. Stake

In [11]:
print(llm_string)

**Background Report: Nvidia’s GTC Conference Announcements and Impact on Competitive Positioning and Long-Term Growth**

**1. Overview of Nvidia's Position in AI and Data Center Markets:**
Nvidia is a leading player in the field of artificial intelligence (AI) and data processing, known for its advancements in graphics processing units (GPUs) that are increasingly being leveraged in AI applications. The company's technology is foundational for various sectors, including automotive, healthcare, data centers, and gaming, positioning Nvidia as a pivotal contributor to the growth and evolution of AI-driven solutions.

**2. The Context of the GTC Conference:**
The GPU Technology Conference (GTC) has evolved from a specialized academic event into a major industry platform spotlighting AI innovations and advancements. Announcements made during the latest conference often set forth Nvidia's strategic directions and highlight new AI product offerings that can shape market dynamics. Stakeholders